# Synthetic Data Generation for Datasheet Extraction

An end-to-end pipeline designed to solve the scarcity of labeled training data in the electronics domain. The system procedurally generates high-fidelity synthetic datasheets (HTML/PDF) paired with structured JSON ground truth. It employs a 'Deep & Wide' strategy, covering both complex multi-condition tables (Active components) and standardized specifications (Passive components), to train state-of-the-art Document Understanding models for automated specification analysis.

In [7]:
import ollama
import json
import random
import os
import datetime
from jinja2 import Template

In [8]:
try:
    # --- שלב 2: שליחת בקשה (Prompt) למודל ---
    MODEL_TO_USE = 'llama3'  # ⚠️ חשוב: שנה לשם של מודל שבאמת מותקן אצלך

    print(f"--- 💬 שולח בקשה למודל: {MODEL_TO_USE} ---")
    
    response = ollama.chat(
        model=MODEL_TO_USE,
        messages=[
            {
                'role': 'user',
                'content': 'כתוב בדיחה קצרה על מתכנתים',
            },
        ]
    )

    print("--- 💡 תשובת המודל ---")
    print(response['message']['content'])

except Exception as e:
    print("\n--- ❌ אירעה שגיאה! ---")
    print(f"פרטי השגיאה: {e}")
    print("\n--- טיפים לפתרון ---")
    print("1. ודא שאפליקציית OLLAMA פועלת ברקע.")
    print(f"2. ודא שהמודל '{MODEL_TO_USE}' באמת מותקן (הר בטרמינל 'ollama list').")
    print("3. אם הוא לא מותקן, הרץ בטרמינל 'ollama pull <model_name>' (למשל 'ollama pull llama3').")

--- 💬 שולח בקשה למודל: llama3 ---

--- ❌ אירעה שגיאה! ---
פרטי השגיאה: model requires more system memory (4.6 GiB) than is available (3.7 GiB) (status code: 500)

--- טיפים לפתרון ---
1. ודא שאפליקציית OLLAMA פועלת ברקע.
2. ודא שהמודל 'llama3' באמת מותקן (הר בטרמינל 'ollama list').
3. אם הוא לא מותקן, הרץ בטרמינל 'ollama pull <model_name>' (למשל 'ollama pull llama3').


# Phase 1 Implementation: Synthetic Generation Pipeline Analysis

In [2]:
# ==========================================
# 1. CONFIGURATION (Design Space)
# ==========================================
OUTPUT_DIR = "output_samples_v2"

STYLES = [
    {"name": "Modern", "font": "Arial, sans-serif", "border": "1px solid #ddd", "header_bg": "#f8f9fa", "padding": "8px"},
    {"name": "Technical", "font": "'Times New Roman', serif", "border": "1px solid black", "header_bg": "#fff", "padding": "4px"},
    {"name": "Compact", "font": "Helvetica, sans-serif", "border": "1px solid #555", "header_bg": "#e0e0e0", "padding": "6px"}
]

# ==========================================
# 2. THE COMPLEX HTML TEMPLATE (Min/Typ/Max + Conditions)
# ==========================================
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <style>
        body { font-family: {{ style.font }}; padding: 30px; }
        .header h1 { margin: 0; font-size: 20px; }
        .header span { color: #666; font-size: 12px; }
        
        table { border-collapse: collapse; width: 100%; margin-top: 25px; font-size: 13px; }
        th, td { border: {{ style.border }}; padding: {{ style.padding }}; vertical-align: top; }
        th { background-color: {{ style.header_bg }}; text-align: left; }
        
        /* עיצוב לתנאים - קטן יותר ולפעמים שבור לשורות */
        .condition-cell { font-size: 0.9em; color: #444; width: 35%; }
        .param-cell { font-weight: bold; width: 25%; }
        .value-cell { text-align: center; width: 8%; }
    </style>
</head>
<body>
    <div class="header">
        <h1>{{ meta.part_number }} - {{ meta.description }}</h1>
        <span>{{ meta.manufacturer }} | {{ meta.date }}</span>
    </div>

    <h3>Electrical Characteristics</h3>
    <p><i>T<sub>A</sub> = 25°C unless otherwise noted.</i></p>

    <table>
        <thead>
            <tr>
                <th>Parameter</th>
                <th>Symbol</th>
                <th>Test Conditions</th>
                <th>Min</th>
                <th>Typ</th>
                <th>Max</th>
                <th>Unit</th>
            </tr>
        </thead>
        <tbody>
            {% for param in specs %}
            <tr>
                <td class="param-cell">{{ param.name }}</td>
                <td>{{ param.symbol }}</td>
                <td class="condition-cell">{{ param.condition | safe }}</td>
                
                <td class="value-cell">{{ param.min if param.min is not none else '-' }}</td>
                <td class="value-cell">{{ param.typ if param.typ is not none else '-' }}</td>
                <td class="value-cell">{{ param.max if param.max is not none else '-' }}</td>
                
                <td>{{ param.unit }}</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
</body>
</html>
"""

# ==========================================
# 3. GENERATOR LOGIC V2
# ==========================================
class DatasheetGeneratorV2:
    def __init__(self):
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

    def _random_value_with_missing(self, base_val, variation=0.1, missing_prob=0.3):
        """ עזר: מחזיר ערך או None בהסתברות מסוימת (מדמה ערך חסר בטבלה) """
        if random.random() < missing_prob:
            return None
        val = base_val * random.uniform(1 - variation, 1 + variation)
        return round(val, 3)

    def generate_mosfet_data(self):
        """ יצירת נתונים מורכבים ל-MOSFET (כמו IRF3205) """
        vds_base = random.choice([30, 55, 60, 100])
        
        specs = []
        
        # 1. Breakdown Voltage (BVDSS) - בדרך כלל יש רק Min
        specs.append({
            "name": "Drain-Source Breakdown Voltage",
            "symbol": "V(BR)DSS",
            "condition": f"Vgs = 0V, Id = 250µA",
            "min": vds_base, "typ": None, "max": None, "unit": "V"
        })

        # 2. Rds(on) - דוגמה לתנאי מורכב
        # לעיתים מופיע פעמיים עבור מתחי שער שונים
        specs.append({
            "name": "Static Drain-Source On-Resistance",
            "symbol": "Rds(on)",
            "condition": "Vgs = 10V, Id = 20A<br>Vgs = 4.5V, Id = 10A", # שבירת שורה ב-HTML
            "min": None, "typ": 0.022, "max": 0.028, "unit": "Ω"
        })

        # 3. Gate Threshold Voltage - טווח מלא (Min/Max)
        specs.append({
            "name": "Gate Threshold Voltage",
            "symbol": "Vgs(th)",
            "condition": "Vds = Vgs, Id = 250µA",
            "min": 2.0, "typ": None, "max": 4.0, "unit": "V"
        })

        # 4. Leakage Current - דוגמה לערכים חסרים (רק Max)
        specs.append({
            "name": "Zero Gate Voltage Drain Current",
            "symbol": "Idss",
            "condition": f"Vds = {vds_base}V, Vgs = 0V",
            "min": None, "typ": None, "max": 1.0, "unit": "µA"
        })

        return {
            "metadata": {
                "part_number": f"IRF-{random.randint(3000, 9999)}",
                "manufacturer": "Synthetic-Semi",
                "description": "N-Channel Power MOSFET",
                "date": str(datetime.date.today())
            },
            "specs": specs
        }

    def render(self, data, style):
        template = Template(HTML_TEMPLATE)
        return template.render(meta=data["metadata"], specs=data["specs"], style=style)

    def save(self, idx, html, data):
        # שמירת HTML
        with open(f"{OUTPUT_DIR}/sample_{idx}.html", "w", encoding="utf-8") as f:
            f.write(html)
        
        # שמירת JSON נקי (Ground Truth)
        # שים לב: המודל יצטרך ללמוד לחלץ את כל המבנה הזה
        clean_json = {"electrical_characteristics": data["specs"]}
        with open(f"{OUTPUT_DIR}/sample_{idx}.json", "w", encoding="utf-8") as f:
            json.dump(clean_json, f, indent=4)
            
        print(f"Generated V2 Sample: {idx}")

    def run(self, count=3):
        print(f"--- Starting Generator V2 (Complex Tables) ---")
        for i in range(count):
            style = random.choice(STYLES)
            data = self.generate_mosfet_data()
            html = self.render(data, style)
            self.save(i, html, data)

if __name__ == "__main__":
    gen = DatasheetGeneratorV2()
    gen.run()

--- Starting Generator V2 (Complex Tables) ---
Generated V2 Sample: 0
Generated V2 Sample: 1
Generated V2 Sample: 2


# VISUAL PLAYGROUND: Single Sample Preview

In [4]:
# 1. יצירת מופע של המנוע הראשי (שהוגדר בתא הקודם)
# אנחנו משתמשים בדיוק באותו Class שייצר את הקבצים
engine = DatasheetGeneratorV2()

# 2. הפעלת הלוגיקה של המנוע
# אנחנו מבקשים מהמנוע: "תייצר לי דאטה" ו-"תרנדר לי HTML"
# שינוי בלוגיקה בתא 1 יופיע כאן מיד!
data = engine.generate_mosfet_data()
style = random.choice(STYLES) # משתמש במשתנה הגלובלי מתא 1
html_output = engine.render(data, style)

# 3. תצוגה
print(f"--- Visual Preview: {data['metadata']['part_number']} ---")
display(HTML(html_output))

print("\n--- Ground Truth JSON (Generated by V2 Logic) ---")
# יצירת מבנה ה-JSON הסופי בדיוק כפי שהוא נשמר לקובץ
clean_json = {"electrical_characteristics": data["specs"]}
print(json.dumps(clean_json, indent=4))

--- Visual Preview: IRF-7014 ---


Parameter,Symbol,Test Conditions,Min,Typ,Max,Unit
Drain-Source Breakdown Voltage,V(BR)DSS,"Vgs = 0V, Id = 250µA",100,-,-,V
Static Drain-Source On-Resistance,Rds(on),"Vgs = 10V, Id = 20AVgs = 4.5V, Id = 10A",-,0.022,0.028,Ω
Gate Threshold Voltage,Vgs(th),"Vds = Vgs, Id = 250µA",2.0,-,4.0,V
Zero Gate Voltage Drain Current,Idss,"Vds = 100V, Vgs = 0V",-,-,1.0,µA



--- Ground Truth JSON (Generated by V2 Logic) ---
{
    "electrical_characteristics": [
        {
            "name": "Drain-Source Breakdown Voltage",
            "symbol": "V(BR)DSS",
            "condition": "Vgs = 0V, Id = 250\u00b5A",
            "min": 100,
            "typ": null,
            "max": null,
            "unit": "V"
        },
        {
            "name": "Static Drain-Source On-Resistance",
            "symbol": "Rds(on)",
            "condition": "Vgs = 10V, Id = 20A<br>Vgs = 4.5V, Id = 10A",
            "min": null,
            "typ": 0.022,
            "max": 0.028,
            "unit": "\u03a9"
        },
        {
            "name": "Gate Threshold Voltage",
            "symbol": "Vgs(th)",
            "condition": "Vds = Vgs, Id = 250\u00b5A",
            "min": 2.0,
            "typ": null,
            "max": 4.0,
            "unit": "V"
        },
        {
            "name": "Zero Gate Voltage Drain Current",
            "symbol": "Idss",
            "c